# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import holoviews as hv


# Import API key
from api_keys import geoapify_key
from holoviews import opts
from bokeh.models import HoverTool

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.77,97,100,15.37,TF,1723762700
1,1,college,64.8569,-147.8028,20.99,44,75,4.12,US,1723762701
2,2,invercargill,-46.4000,168.3500,8.34,86,100,0.45,NZ,1723762703
3,3,avarua,-21.2078,-159.7750,22.03,56,20,4.12,CK,1723762704
4,4,fuling,29.7022,107.3919,24.68,87,14,0.44,CN,1723762707


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points( 
    "Lng",
    "Lat",   
    geo= True,      
    tiles = "OSM",  
    frame_width = 800, 
    frame_height = 600, 
    size = "Humidity",
    scale = 1, 
    color = "City"

)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & 
                                (city_data_df["Wind Speed"] < 4.5) &  
                                 (city_data_df["Cloudiness"] == 0), :]

# Drop any rows with null values
ideal_cities.dropna


# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,emerald,-23.5333,148.1667,22.61,55,0,3.89,AU,1723762757
36,36,jamestown,42.0970,-79.2353,26.80,44,0,0.00,US,1723762762
51,51,salinas,36.6777,-121.6555,22.13,74,0,3.09,US,1723762788
77,77,jatai,-17.8814,-51.7144,25.81,22,0,1.86,BR,1723762848
82,82,chenaran,36.6455,59.1212,23.39,56,0,0.47,IR,1723762856
141,141,ulmeni,47.4667,23.3000,22.30,56,0,1.05,RO,1723762966
182,182,constantia,44.1833,28.6500,21.50,61,0,4.39,RO,1723763030
207,207,cine,37.6117,28.0614,23.67,30,0,0.72,TR,1723763082
220,220,kone,-21.0595,164.8658,26.94,49,0,4.25,NC,1723763108
289,289,chibougamau,49.9168,-74.3659,26.79,44,0,2.57,CA,1723762936


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,emerald,AU,-23.5333,148.1667,55,None
36,jamestown,US,42.0970,-79.2353,44,None
51,salinas,US,36.6777,-121.6555,74,None
77,jatai,BR,-17.8814,-51.7144,22,None
82,chenaran,IR,36.6455,59.1212,56,None
141,ulmeni,RO,47.4667,23.3000,56,None
182,constantia,RO,44.1833,28.6500,61,None
207,cine,TR,37.6117,28.0614,30,None
220,kone,NC,-21.0595,164.8658,49,None
289,chibougamau,CA,49.9168,-74.3659,44,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 1 
categories = f"accommodation.hotel"
params = {
   "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
   

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
emerald - nearest hotel: Emerald Central Hotel
jamestown - nearest hotel: DoubleTree Jamestown
salinas - nearest hotel: The Traveler's Hotel
jatai - nearest hotel: Hotel Rio Claro
chenaran - nearest hotel: No hotel found
ulmeni - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
cine - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
chibougamau - nearest hotel: No hotel found
biloela - nearest hotel: Settlers Hotel Motel
colorado - nearest hotel: Servilha Park Hotel
hazorasp - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
lucheng - nearest hotel: 富华山庄
karratha - nearest hotel: Karratha International Hotel
aksu - nearest hotel: Akesu Tianyuan International Hotel
madalena - nearest hotel: Hotel de Paredes
eastend - nearest hotel: Cypress Hotel
lappersdorf - nearest hotel: Hotel Bartholomäus
strazhitsa - nearest hotel: Чаир


,City,Country,Lat,Lng,Humidity,Hotel Name
33,emerald,AU,-23.5333,148.1667,55,Emerald Central Hotel
36,jamestown,US,42.0970,-79.2353,44,DoubleTree Jamestown
51,salinas,US,36.6777,-121.6555,74,The Traveler's Hotel
77,jatai,BR,-17.8814,-51.7144,22,Hotel Rio Claro
82,chenaran,IR,36.6455,59.1212,56,No hotel found
141,ulmeni,RO,47.4667,23.3000,56,No hotel found
182,constantia,RO,44.1833,28.6500,61,Hotel Dali
207,cine,TR,37.6117,28.0614,30,No hotel found
220,kone,NC,-21.0595,164.8658,49,Pacifik Appartels
289,chibougamau,CA,49.9168,-74.3659,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.


In [12]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points( 
    "Lng",
    "Lat",   
    geo= True,      
    tiles = "OSM",  
    frame_width = 800, 
    frame_height = 600, 
    size = "Humidity",
    scale = 1, 
    color = "City",
    hover_cols =["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
city_hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)